<a href="https://colab.research.google.com/github/yeomyeom/2019_cau_oss_hackathon/blob/master/hackathon_team06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
 *  단, 사용할 데이터셋에 따라 is_mnist만 수정
*   모든 구현은 [2. 데이터 전처리]와 [3. 모델 생성]에서만 진행
 *  데이터 전처리 후 트레이닝, 데이터 셋은 x_train_after, x_test_after 변수명을 유지해주세요.
 *  데이터셋이 달라져도 같은 모델 구조를 유지하여야함.
*   [4. 모델 저장]과 [5. 모델 로드 및 평가]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *  트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
 *  team_name을 제외한 다른 부분은 수정하지 말 것
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  두 개의 데이터셋에 대해 다른 모델 구조를 사용한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

is_mnist = False;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**


In [63]:
'''
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0
if is_mnist == True:
  input_shape = (x_train.shape[1],x_train.shape[2],1)
  x_train_after = np.reshape(x_train_after,(60000,28,28,1))
  x_test_after = np.reshape(x_test_after,(10000,28,28,1)) 
'''
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0
if is_mnist == True:
  input_shape = (x_train.shape[1],x_train.shape[2],1)
  x_train_after = np.reshape(x_train_after,(60000,28,28,1))
  x_test_after = np.reshape(x_test_after,(10000,28,28,1))
  
  
####  
# Standardize images across the dataset, mean=0, stdev=1
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras import backend as K
K.set_image_dim_ordering('th')

# convert from int to float
x_train_after = x_train_after.astype('float32')
x_test_after = x_test_after.astype('float32')
# define data preparation

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
#datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
#datagen = ImageDataGenerator(zca_whitening=True)

# fit parameters from data
datagen.fit(x_train_after)
# configure batch size and retrieve one batch of images

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:940: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (50000, 32, 32, 3) (32 channels).
  ' channels).')


# **3. 모델 생성**



In [64]:
'''
from keras import optimizers

model = keras.Sequential()

model.add(keras.layers.Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape = input_shape))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(1500, activation='relu'))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
#adam = optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-8, decay=0.0, amsgrad=False)

save_path = '/content/'
team_name = 'team06'
cd_checkpoint = keras.callbacks.ModelCheckpoint(filepath=save_path +  'model_entire_' + data_type + '_' + team_name + '.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weight_only=True)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 100, epochs = 100, shuffle=True, validation_data=[x_test_after, y_test], callbacks=[cd_checkpoint])

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
print('model recall')
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')
print('weight save')
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)
print('json save')
'''

"\nfrom keras import optimizers\n\nmodel = keras.Sequential()\n\nmodel.add(keras.layers.Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape = input_shape))\nmodel.add(keras.layers.BatchNormalization())\nmodel.add(keras.layers.Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))\nmodel.add(keras.layers.BatchNormalization())\nmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))\nmodel.add(keras.layers.Dropout(0.2))\n\nmodel.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))\nmodel.add(keras.layers.BatchNormalization())\nmodel.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))\nmodel.add(keras.layers.BatchNormalization())\nmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))\nmodel.add(keras.layers.Dropout(0.3))\n\nmodel.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))\nmodel.a

In [0]:
from keras import optimizers
from keras.layers import LeakyReLU

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_initializer='he_normal', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, kernel_initializer='he_normal', kernel_size=(3,3), strides=(1,1), padding='same', input_shape = input_shape))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(64, kernel_initializer='he_normal', kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Conv2D(64, kernel_initializer='he_normal', kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Conv2D(128, kernel_initializer='he_normal', kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(500, kernel_initializer='he_normal'))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(keras.layers.Dense(num_classes, kernel_initializer='he_normal', activation='softmax'))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
adam = optimizers.Adam(lr=0.005,beta_1=0.9,beta_2=0.999,epsilon=None, decay=1e-6, amsgrad=False)

save_path = '/content/'
team_name = 'team06'
cd_checkpoint = keras.callbacks.ModelCheckpoint(filepath=save_path +  'model_entire_' + data_type + '_' + team_name + '.h5', monitor='val_acc', verbose=1, save_best_only=True)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', learning_rate=0.001, optimizer='adam', metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 100, epochs = 200, shuffle=True, validation_data=[x_test_after, y_test], callbacks=[cd_checkpoint])
model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
print('model recall')
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')
print('weight save')
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)
print('json save')

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
49800/50000 [============================>.] - ETA: 0s - loss: 0.0810 - acc: 0.3591
Epoch 00001: val_acc improved from -inf to 0.45010, saving model to /content/model_entire_cifar10_team06.h5
50000/50000 [==============================] - 21s 419us/sample - loss: 0.0810 - acc: 0.3596 - val_loss: 0.0726 - val_acc: 0.4501
Epoch 2/200
49800/50000 [============================>.] - ETA: 0s - loss: 0.0628 - acc: 0.5114
Epoch 00002: val_acc improved from 0.45010 to 0.58280, saving model to /content/model_entire_cifar10_team06.h5
50000/50000 [==============================] - 8s 168us/sample - loss: 0.0628 - acc: 0.5117 - val_loss: 0.0567 - val_acc: 0.5828
Epoch 3/200
49600/50000 [============================>.] - ETA: 0s - loss: 0.0555 - acc: 0.5775
Epoch 00003: val_acc improved from 0.58280 to 0.59640, saving model to /content/model_entire_cifar10_team06.h5
50000/50000 [==============================] - 8s 168us/sample - loss: 0.

In [0]:
'''
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Flatten layer: 28 x 28 x 1 image를 784개의 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=input_shape))

# 1st hidden layer: fully-connected layer
# (# of inputs = 784, # of outputs = 512, actication fuction = relu)
model.add(keras.layers.Dense(512, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 512, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(64, activation=tf.nn.relu))

# 4rd hidden layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 32, actication fuction = relu)
model.add(keras.layers.Dense(32, activation=tf.nn.relu))

# 5rd hidden layer: fully-connected layer 
# (# of inputs = 32, # of outputs = 16, actication fuction = relu)
model.add(keras.layers.Dense(16, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 16, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 30, shuffle=True, validation_data=[x_test_after, y_test])
'''

# **4. 모델 저장**

In [56]:

save_path = '/content/'
team_name = 'team06'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')


"\nsave_path = '/content/'\nteam_name = 'team06'\n\n# 모델의 weight 값만 저장합니다.\nmodel.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')\n\n# 모델의 구조만을 저장합니다.\nmodel_json = model.to_json()\nwith open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : \n    json_file.write(model_json)\n\n# 트레이닝된 전체 모델을 저장합니다.\nmodel.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')\n"

# **5. 모델 로드 및 평가**

In [61]:
save_path = '/content/'
team_name = 'team06'

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
model.summary()
model.evaluate(x_test_after, y_test)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_160 (Conv2D)          (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization_161 (Bat (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_161 (Conv2D)          (None, 32, 32, 16)        2320      
_________________________________________________________________
batch_normalization_162 (Bat (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d_135 (MaxPoolin (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_120 (Dropout)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_162 (Conv2D)          (None, 16, 16, 32)      

[0.02246283275745809, 0.8502]